In [9]:
import numpy as np
import random
from random import sample

# Sample input adjacency graph
arr = [[0,1,0,0,0,0], [1,0,1,0,0,0],[0,1,0,1,1,0],[0,0,1,0,1,0],[0,0,1,1,0,1],[0,0,0,0,1,0]]
graph = np.array(arr)
graph

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 1, 0]])

In [10]:
# Function to return the probabilty of an actual infection sequence, not the observed one
# Input is adjacency graph, observed sequence, actual infection sequence, and given probability p
# Return p^(number not observed, i.e asymptomatic) times (1-p)^(number observed, i.e symptomatic)
def asymptomatic_probability(graph, observed_seq, actualinfection_seq, p):
    number_not_observed = len(actualinfection_seq) - len(observed_seq)
    number_observed = len(observed_seq)
    probability = p**(number_not_observed) * (1-p)**(number_observed)
    return probability

# Test with above graph and actual sequence [1,2,3,4,5] and observed sequence [2,4,5] and p = 0.4
asymptomatic_probability(graph, [2,4,5], [1,2,3,4,5], 0.4)

0.03456

In [11]:
# This cell is exclusively for helper functions for the asymptomatic sequence simulation

# This helper function returns an array of a predicted asymptomatic sequence but remove all the asymptomatic
# that is not observed nodes
def helper_findSubList(predicit_asymptomatic_seq, not_observed):
    copy_arr = predicit_asymptomatic_seq.copy()
    for i in not_observed:
        if i in copy_arr:
            copy_arr.remove(i)
    return copy_arr

# This global variable is used to check whether the predicted asymptomatic sequence is legal
valid = False

# This helper function (along with the below) helps validate the predicted asymptomatic sequence
def helper_validate_seq_1(graph, predicit_asymptomatic_seq):
    # First set up the global varibale valid to be false, which if is legal then modify it using above helper
    global valid
    valid = False
    # Feed index and validate into recursion
    helper_validate_seq_2(graph, predicit_asymptomatic_seq, 1, False)
    
def helper_validate_seq_2(graph, predicit_asymptomatic_seq, index, validate):
    validate_copy = validate
    # Base case, if reach the end index node of the predicted asymptomatic sequence then modify global variable
    # valid to be True and the sequence is legal
    if index == len(predicit_asymptomatic_seq):
        global valid 
        valid = True
        return
    # Itertae from the first index to the last index to see if there is a path
    # If there is, the sequence is legal
    for i in range(index):
        if graph[predicit_asymptomatic_seq[i]][predicit_asymptomatic_seq[index]] == 1:
            validate_copy = True
    # If there is no path, then the sequence is illgeal
    if validate_copy == False:
        return
    # Decrement index until index becomes 0 that is base case
    index = index + 1
    helper_validate_seq_2(graph, predicit_asymptomatic_seq, index, False)

# Test some predicted asymptomatic sequences
helper_validate_seq_1(graph, [1,2,3,5])
print(valid)
helper_validate_seq_1(graph, [1,2,4,5])
print(valid)
helper_validate_seq_1(graph, [1,0,2,3])
print(valid)
helper_validate_seq_1(graph, [1,0,2,4,3])
print(valid)
helper_validate_seq_1(graph, [1,0,2,4])
print(valid)
helper_validate_seq_1(graph, [1,0,2,3,5])
print(valid)
helper_validate_seq_1(graph, [1,0,2,3,4])
print(valid)
helper_validate_seq_1(graph, [1,0,2,5,4,3])
print(valid)

False
True
True
True
True
False
True
False


In [12]:
# Helper function to find possible added nodes to be concanteated with the origin sequence
# Use recursion to find nodes that is connected to the started nodes,i.e there is a path
# Return the possible added nodes
def possible_added_nodes(graph, start_nodes_ls, rest_of_nodes, count, result):
    # Base case, if we iterate through all the node then done
    if count == len(rest_of_nodes):
        return result
    # Iterate through the start nodes with the rest of the nodes
    # If there is path, i.e == 1, append them to the result
    for i in start_nodes_ls:
        for j in rest_of_nodes:
            if graph[i][j] == 1 and j not in result:
                result.append(j)
                start_nodes_ls.append(j)
    count = count + 1
    return possible_added_nodes(graph, start_nodes_ls, rest_of_nodes, count, result)

# Testing with start node = [3,2] and rest of nodes = [4,5]
possible_added_nodes(graph, [3,2], [4, 5], 0, [])

[4, 5]

In [13]:
# Function to generate a predicted asymptomatic sequence
# Input is graph and observed sequence array
# Return a valid predicted asymptomatic sequence
def asymptomatic_simulation(graph, orig_sequence, observed_set):
    # Get all nodes in graph
    all_nodes = [item for item in range(len(graph))]
    # Get nodes that is part of observed set but not part of the origin sequence
    sub_observed_set = list(set(observed_set) - set(orig_sequence))
    # Get the rest of the nodes not part of origin seq or observed set
    rest_of_nodes = list(set(all_nodes) - set(orig_sequence) - set(observed_set))
    # Get the starting nodes which include the last element of the origin sequence 
    # and the subset of oberserved nodes
    start_nodes_ls = orig_sequence + sub_observed_set
    # Use helper methods to find possible nodes to be added to the origin sequence
    added_nodes = possible_added_nodes(graph, start_nodes_ls, rest_of_nodes, 0, [])
    # Keep generating random predicted sequence until a valid one is found
    while True:
        # Get a random limit of added nodes
        random_limit = random.randint(1, len(added_nodes))
        # Get the sample nodes based on the random limit
        sample_added_node = sample(added_nodes,random_limit)
        # Put together the added nodes with the sub observed node set
        concantenate_nodes = sample_added_node + sub_observed_set
        # Random suffle the sequence
        random.shuffle(concantenate_nodes)
        # Now put the origin suquence in front of the generated shuffled sequence
        predicit_asymptomatic_seq = orig_sequence + concantenate_nodes
        # Use helper method to validate the sequence
        helper_validate_seq_1(graph, predicit_asymptomatic_seq)
        if valid == True:
            return predicit_asymptomatic_seq
        
# Test with above graph with sequence [1,0,2] and observed set = [1,2,3] 
print(asymptomatic_simulation(graph, [1,0,2], [1,2,3]))
print(asymptomatic_simulation(graph, [1,0,2], [1,2,3]))
print(asymptomatic_simulation(graph, [1,0,2], [1,2,3]))
print(asymptomatic_simulation(graph, [1,0,2], [1,2,3]))
print(asymptomatic_simulation(graph, [1,0,2], [1,2,3]))

[1, 0, 2, 4, 3]
[1, 0, 2, 4, 3]
[1, 0, 2, 4, 3]
[1, 0, 2, 3, 4, 5]
[1, 0, 2, 3, 4]


In [14]:
# Test with another graph
arr = [[0,1,0,0,0,0], [1,0,1,1,0,0],[0,1,0,1,0,0],[0,1,1,0,1,1],[0,0,0,1,0,0],[0,0,0,1,0,0]]
graph = np.array(arr)
graph

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [15]:
# Test with above graph sequence [1,2] and observed set = [2,5]       
print(asymptomatic_simulation(graph, [1,2], [2,5]))
print(asymptomatic_simulation(graph, [1,2], [2,5]))
print(asymptomatic_simulation(graph, [1,2], [2,5]))
print(asymptomatic_simulation(graph, [1,2], [2,5]))
print(asymptomatic_simulation(graph, [1,2], [2,5]))

[1, 2, 0, 3, 5]
[1, 2, 3, 0, 5, 4]
[1, 2, 3, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 5]
